In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
%matplotlib inline

In [257]:
pd.set_option('display.max_columns', None)

In [258]:
df = pd.read_csv('../AdventureWorks2019withBikeNames.csv', sep=';', header=0)

In [259]:
df['CatName'].unique()

array(['Bikes', 'Clothing', 'Accessories', 'Components'], dtype=object)

In [260]:
df['SubName'][df['CatName'] == 'Bikes'].unique()

array(['Mountain Bikes', 'Road Bikes', 'Touring Bikes'], dtype=object)

In [261]:
df[['StandardCost', 'UnitPrice', 'UnitPriceDiscount', 'SubTotal']].head(2)

,StandardCost,UnitPrice,UnitPriceDiscount,SubTotal
0,"1898,0944","2024,994","0,00","20565,6206"
1,"1898,0944","2024,994","0,00","20565,6206"


The values in table has wrong format. I need to change , for . and change format to float

In [262]:
df['UnitPrice'] = df[['UnitPrice']].applymap(lambda x: str(x.replace(',','.'))).astype('float')

In [263]:
df['StandardCost'] = df[['StandardCost']].applymap(lambda x: str(x.replace(',','.'))).astype('float')
df['UnitPriceDiscount'] = df[['UnitPriceDiscount']].applymap(lambda x: str(x.replace(',','.'))).astype('float')
df['SubTotal'] = df[['SubTotal']].applymap(lambda x: str(x.replace(',','.'))).astype('float')

In [264]:
df['ListPrice'] = df[['ListPrice']].applymap(lambda x: str(x.replace(',','.'))).astype('float')

#### Last order has been done on 2014-06-30

In [265]:
from dateutil import relativedelta


In [266]:
from datetime import datetime

In [267]:
df['SellStartDate'] =  pd.to_datetime(df['SellStartDate'])

In [268]:
df['SellEndDate'] =  pd.to_datetime(df['SellEndDate'])

I want to count how many mounths bikes is or has benn aveliable to buy

In [270]:
from dateutil import relativedelta

In [271]:
last_order = '2014-06-30'
last_order = pd.to_datetime(last_order)

In [277]:
pd.isnull(df['SellEndDate'].iloc[22])

True

In [281]:
df['difM'] = 0

In [282]:
for i in range(len(df)):
    if pd.isnull(df['SellEndDate'].iloc[i]) == False:
        df['difM'].iloc[i] = round((relativedelta.relativedelta(df['SellEndDate'][i], df['SellStartDate'][i])).months + (relativedelta.relativedelta(df['SellEndDate'][i], df['SellStartDate'][i])).years * 12 + ((relativedelta.relativedelta(df['SellEndDate'][i], df['SellStartDate'][i])).days)/30, 0)
    else:
        df['difM'].iloc[i] = round((relativedelta.relativedelta(last_order, df['SellStartDate'][i])).months + (relativedelta.relativedelta(last_order, df['SellStartDate'][i])).years * 12 + ((relativedelta.relativedelta(last_order, df['SellStartDate'][i])).days)/30, 0)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [284]:
df['difM'].unique()

array([12, 37, 24, 25, 13], dtype=int64)

In [286]:
df['SellStartDate'].unique()

array(['2011-05-31T00:00:00.000000000', '2012-05-30T00:00:00.000000000',
       '2013-05-30T00:00:00.000000000'], dtype='datetime64[ns]')

As we can see the shop has 5 different period of aveliability in store. We know that first bike ship in 2011 and the lat oreder is from 2014, so its 4 years of existing. We can also conclude that shop launch new models once a year

I would like to split products by 3 periods categories:
- around 1 year
- around 2 years
- around 3 years

In [291]:
len(df[df['difM'] <= 13])

74848

In [295]:
len(df[(df['difM'] >= 24) & (df['difM'] <= 25)])

26198

In [296]:
len(df[df['difM'] >= 37])

20271

We must keep in mind that some of the bikes are no longer aveliable also and they can cluster in each 3 groups. That is why we should split them also by aviability. The question is if it's better to make 2 analysis or split them into 6 group. The big advantage is that I would like to split them also by subcategory so in the end I will get 18 groups to analyze

-----------------------------------------------
I decided that I will not split them by 3 periods of aveliability

In [299]:
one_y_bikes = df[df['difM'] <= 13].copy()

In [300]:
two_y_bikes = df[(df['difM'] >= 24) & (df['difM'] <= 25)].copy()
three_y_bikes = df[df['difM'] >= 37].copy()

In [301]:
3/5

0.6

In [305]:
300/250

1.2

## 3 subcategory of bikes:

In [46]:
RoadBikes = df[df['SubName'] == 'Road Bikes']
MountainBikes = df[df['SubName'] == 'Mountain Bikes']
TouringBikes = df[df['SubName'] == 'Touring Bikes']

## Spliting bikes

Firt of all I would like to split bikes by avebility, becouse some of the bikes are no longer aveliable in store. I will concat them in the end to compare

In [47]:
# Unaveliable car has SellEndDate, so if it is not null it means that the bike is o longer avliable to buy
UnaveliableMB = MountainBikes[MountainBikes['SellEndDate'].isnull() == False]
UnaveliableRB = RoadBikes[RoadBikes['SellEndDate'].isnull() == False]
UnaveliableTB = TouringBikes[TouringBikes['SellEndDate'].isnull() == False]

In [48]:
# analogously
AveliableMB = MountainBikes[MountainBikes['SellEndDate'].isnull() == True]
AveliableRB = RoadBikes[RoadBikes['SellEndDate'].isnull() == True]
AveliableTB = TouringBikes[TouringBikes['SellEndDate'].isnull() == True]

### Question 1
###### How many bicykles shop has now and how many are no longer aveliable in each subcategory? 

In [49]:
x1 = len(UnaveliableMB.groupby('ProductID').count())
x2 = len(UnaveliableRB.groupby('ProductID').count())
x3 = len(UnaveliableTB.groupby('ProductID').count())

x11 = len(AveliableMB.groupby('ProductID').count())
x12 = len(AveliableRB.groupby('ProductID').count())
x13 = len(AveliableTB.groupby('ProductID').count())


print(f'There is {x1} unaveliable Mountain bikes')
print(f'There is {x2} unaveliable Road bikes')
print(f'There is {x3} unaveliable Touring bikes')
print('------------------------------------------')
print(f'There is {x11} unaveliable Mountain bikes')
print(f'There is {x12} unaveliable Road bikes')
print(f'There is {x13} unaveliable Touring bikes')

There is 12 unaveliable Mountain bikes
There is 25 unaveliable Road bikes
There is 0 unaveliable Touring bikes
------------------------------------------
There is 20 unaveliable Mountain bikes
There is 18 unaveliable Road bikes
There is 22 unaveliable Touring bikes


Intresting thing is that there is no unaveliable Touring bikes. It means that it might be new Subcategory of bikes in the Store. It looks like company has given up Road bicycles in favor of Touring bikes.

In [50]:
UnaveliableRB['ProductID'].unique()

array([762, 758, 764, 770, 754, 765, 768, 753, 756, 763, 749, 760, 766,
       755, 761, 767, 750, 751, 752, 759, 769, 757, 790, 789, 791],
      dtype=int64)

In [51]:
AveliableTB['ProductID'].unique()

array([957, 969, 972, 954, 965, 964, 958, 970, 967, 956, 978, 966, 961,
       953, 955, 968, 971, 963, 979, 959, 962, 960], dtype=int64)

### Question 2

###### Which bikes are the most/the least ordered in each grups (6)?

In [118]:
# I count sum of orders for each ProductID
QuantUnavMB = UnaveliableMB[['OrderQty', 'ProductID']].groupby('ProductID').sum().sort_values('ProductID', ascending=False)

In [119]:
QuantUnavMB['SubName'] = 'MountainBikes'
QuantUnavMB['Aveliablility'] = False

In [120]:
QuantUnavRB = UnaveliableRB[['OrderQty', 'ProductID']].groupby('ProductID').sum().sort_values('ProductID', ascending=False)
QuantUnavRB['SubName'] = 'RoadBikes'
QuantUnavRB['Aveliablility'] = False

# There is no Touring bikes unavelible so I will skip that

In [121]:
QuantAvMB = AveliableMB[['OrderQty', 'ProductID']].groupby('ProductID').sum().sort_values('ProductID', ascending=False)
QuantAvMB['SubName'] = 'MountainBikes'
QuantAvMB['Aveliablility'] = True

QuantAvRB = AveliableRB[['OrderQty', 'ProductID']].groupby('ProductID').sum().sort_values('ProductID', ascending=False)
QuantAvRB['SubName'] = 'RoadBikes'
QuantAvRB['Aveliablility'] = True

QuantAvTB = AveliableTB[['OrderQty', 'ProductID']].groupby('ProductID').sum().sort_values('ProductID', ascending=False)
QuantAvTB['SubName'] = 'TouringBikes'
QuantAvTB['Aveliablility'] = True

###### Best selling

In [131]:
QuantUnavMB.sort_values('OrderQty', ascending=False).head(2)

,OrderQty,SubName,Aveliablility
ProductID,,,
786,776,MountainBikes,False
787,747,MountainBikes,False


In [132]:
QuantUnavRB.sort_values('OrderQty', ascending=False).head(2)

,OrderQty,SubName,Aveliablility
ProductID,,,
770,2270,RoadBikes,False
762,2254,RoadBikes,False


In [133]:
QuantAvMB.sort_values('OrderQty', ascending=False).head(2)

,OrderQty,SubName,Aveliablility
ProductID,,,
782,2977,MountainBikes,True
783,2664,MountainBikes,True


In [134]:
QuantAvRB.sort_values('OrderQty', ascending=False).head(2)

,OrderQty,SubName,Aveliablility
ProductID,,,
801,1763,RoadBikes,True
797,1744,RoadBikes,True


In [135]:
QuantAvTB.sort_values('OrderQty', ascending=False).head(2)

,OrderQty,SubName,Aveliablility
ProductID,,,
969,1120,TouringBikes,True
957,1114,TouringBikes,True


Conclusion TOP sales Bikes unaveliable belongs deffinitly to RoadBikes. First top sell bike from this group has been sold more than 2270 times. If we look at only aveliable bikes we can se that this reliance has split for top Road and Toring bikes. We can to say much more untill we compare quantity to income or ROI. Letter I will try to find out what the best and worst selling bikes has in common.

###### Worst selling

In [136]:
QuantUnavMB.sort_values('OrderQty', ascending=True).head(2)

,OrderQty,SubName,Aveliablility
ProductID,,,
774,505,MountainBikes,False
772,593,MountainBikes,False


In [138]:
QuantUnavRB.sort_values('OrderQty', ascending=True).head(2)

,OrderQty,SubName,Aveliablility
ProductID,,,
757,156,RoadBikes,False
756,346,RoadBikes,False


In [140]:
QuantAvMB.sort_values('OrderQty', ascending=True).head(2)

,OrderQty,SubName,Aveliablility
ProductID,,,
993,272,MountainBikes,True
989,282,MountainBikes,True


In [141]:
QuantAvRB.sort_values('OrderQty', ascending=True).head(2)

,OrderQty,SubName,Aveliablility
ProductID,,,
975,536,RoadBikes,True
997,656,RoadBikes,True


In [142]:
QuantAvTB.sort_values('OrderQty', ascending=True).head(2)

,OrderQty,SubName,Aveliablility
ProductID,,,
960,281,TouringBikes,True
978,292,TouringBikes,True


If we look at worst selling bikes we can see that even the worst selling road bikes doing quite well in both av/unave groups, specially now. We can see also that new MB do not sell the best compered to the old one. This may be due to late start of the sale of bike

### Question 3

##### Which bikes gives the best income? (Without cross selling)

In [ ]:
QuantUnavMB